In [1]:
#!/usr/bin/env python3
import os
import re
import csv
import logging
import argparse
from pathlib import Path
from collections import defaultdict

import sys

In [2]:
sys.path.append(os.path.abspath('/home/asmodi/Code/git/markdown_linker/src/'))
from Backlink import *

## Sys Functions

In [3]:
def __init__():
    sys_dict = generate_sys_dict()
    md_list = generate_link_list(sys_dict['SYSTEM_PATH'])
    sys_dict = populate_markdown_dictionary(md_list, sys_dict)
    sys_dict = markdown_crossrefrence(sys_dict)
    return sys_dict

In [4]:
system_dict  = __init__()

In [5]:
system_dict['MARKDOWNS_DICT']['/SlipBox/AI_Levels_20280826.md']['PATH']

PosixPath('/home/asmodi/Code/git/markdown_linker/test/markdown/SlipBox/AI_Levels_20280826.md')

In [6]:
system_dict['MARKDOWNS_DICT']['/SlipBox/AI_Levels_20280826.md']

{'PATH': PosixPath('/home/asmodi/Code/git/markdown_linker/test/markdown/SlipBox/AI_Levels_20280826.md'),
 'REL_PATH': '/SlipBox/AI_Levels_20280826.md',
 'BACKLINKS': [('Static Axioms and Dynamic Policies',
   '/SlipBox/BetterThanHuman_or_BetterThanHumanAndMachine_20250826/Static_Axioms_and_Dynamic_Policies.md')],
 'BACKLINKS_PATH': ['/SlipBox/BetterThanHuman_or_BetterThanHumanAndMachine_20250826/Static_Axioms_and_Dynamic_Policies.md'],
 'LINKS': [],
 'LINKS_PATH': [],
 'NEED2UPDATE': False,
 'ID': None,
 'DESCRIPTION': '',
 'TAGS': ['purpose', ' system applicaitons'],
 'EDITOR': 'markdown',
 'TITLE': 'AI Levels',
 'DATECREATED': '2025-08-27T02:00:35.044Z',
 'PUBLISHED': 'true',
 'DATE': '2025-08-27T02:00:35.044Z'}

# Internal Logic

# Loading Tests

In [21]:
def generate_backlink_section(markdown_dict):
    txt_list = []
    for title, lnk in markdown_dict['BACKLINKS_PATH']:
        txt_list.append(f'- [{title}]({lnk})')

    txt = '\n# Backlinks\n\n' + '\n'.join(txt_list)
    return(txt)

In [25]:
def update_markdown_file(markdown_dic, FixLists):
    markdown_content = read_markdown_doc(markdown_dic['PATH'])

    # step 1 fix backlinks
    if markdown_dic['REL_PATH'] in FixLists.get('BACKLINKS',''):
        split_md = split_on_backlinks_section(markdown_content)
        split_md = split_md[0]
        backlinks_section = generate_backlink_section(markdown_dic)
        markdown_content = split_md + backlinks_section
        FixLists['BACKLINKS'].remove(markdown_dic['REL_PATH'])
    # step 2 - add anything else that needs to be fixed

    # step 3 - save
    write_markdown_doc(markdown_dic['PATH'], markdown_content)

In [ ]:
def fix_markdowns(system_dic):
    markdown_2_update = list(set(sum(system_dic.get('UPDATE', {}).values(), [])))
    for md in markdown_2_update:
        update_markdown_file(system_dic['MARKDOWNS_DICT'][md], system_dic['UPDATE'])

In [50]:
MARKDOWN_FILE = read_markdown_doc(system_dict['MARKDOWNS_DICT']['/SlipBox/AI_Levels_20280826.md']['PATH'])

In [51]:
MARKDOWN_FILE

'---\ntitle: AI Levels\ndescription: \npublished: true\ndate: 2025-08-27T02:00:35.044Z\ntags: purpose, system applicaitons\neditor: markdown\ndateCreated: 2025-08-27T02:00:35.044Z\n---\n\n# AI levels\n\n**Unique Id:** c2b858a4fcc1eb9d9e85490553e63c85 \n#sysemApplication\n#Purpose\n\n----\n\nThere are at least three different levels to AI. Their is the backend level, which either allows for the use of "code vibing" or enhancment of the development space, then the front end, which is what a customer interacts with: Think the backstreet boys having the floating heads move and "sync" to the actual singers. Then there is the middle ground, is unclear to me what of that that is.\n\nIn each cases, the application, **end goal**, and money path is different.\n\n# '

In [52]:
system_dict['UPDATE']['BACKLINKS'] = system_dict['UPDATE']['backlinks'].copy()

In [53]:
generate_backlink_section(system_dict['MARKDOWNS_DICT']['/SlipBox/AI_Levels_20280826.md'])

'\n# Backlinks\n\n- [Static Axioms and Dynamic Policies](/SlipBox/BetterThanHuman_or_BetterThanHumanAndMachine_20250826/Static_Axioms_and_Dynamic_Policies.md)'

In [54]:
update_markdown_file(system_dict['MARKDOWNS_DICT']['/SlipBox/AI_Levels_20280826.md'],system_dict['UPDATE'])

In [55]:
read_markdown_doc(system_dict['MARKDOWNS_DICT']['/SlipBox/AI_Levels_20280826.md']['PATH'])

'---\ntitle: AI Levels\ndescription: \npublished: true\ndate: 2025-08-27T02:00:35.044Z\ntags: purpose, system applicaitons\neditor: markdown\ndateCreated: 2025-08-27T02:00:35.044Z\n---\n\n# AI levels\n\n**Unique Id:** c2b858a4fcc1eb9d9e85490553e63c85 \n#sysemApplication\n#Purpose\n\n----\n\nThere are at least three different levels to AI. Their is the backend level, which either allows for the use of "code vibing" or enhancment of the development space, then the front end, which is what a customer interacts with: Think the backstreet boys having the floating heads move and "sync" to the actual singers. Then there is the middle ground, is unclear to me what of that that is.\n\nIn each cases, the application, **end goal**, and money path is different.\n\n# \n# Backlinks\n\n- [Static Axioms and Dynamic Policies](/SlipBox/BetterThanHuman_or_BetterThanHumanAndMachine_20250826/Static_Axioms_and_Dynamic_Policies.md)'

# Old Docs

In [ ]:

def scan_documents(scan_path):
    """Scan all markdown files and build comprehensive link data"""
    scan_path = Path(scan_path).resolve()
    logging.info(f"Scanning documents in {scan_path}")
    csv_path = scan_path / 'backlinks.csv'
    # existing_data = load_csv_data(csv_path)
    

    markdown_header = {} # Map of file path to its header/title
    links_data = []
    backlinks_map = defaultdict(set)
    md_files = list(scan_path.rglob('*.md'))
    
    logging.info(f"Found {len(md_files)} markdown files")
    
    for md_file in md_files:
        content = read_markdown_doc(md_file)
            
        links_found = find_markdown_links(content)
        if links_found:
            logging.debug(f"Found {len(links_found)} links in {md_file.name}")
        print(links_found)    
        for link_text, target_file in links_found:
            # Handle links relative to scan directory
            if target_file.startswith('/'):
                # Link is scan-relative (e.g., /TESTDIR/docs/file.md)
                target_parts = Path(target_file).parts[1:]  # Remove leading '/'
                if target_parts and target_parts[0].upper() == scan_path.name.upper():
                    # Link points within scan structure - convert to absolute path
                    rel_path = Path(*target_parts[1:]) if len(target_parts) > 1 else Path('.')
                    target_path = (scan_path / rel_path).resolve()
                    logging.debug(f"Resolved scan-relative link {target_file} to {target_path}")
                else:
                    # Link points outside scan structure
                    target_path = Path(target_file)
            else:
                # Link is relative to current file location
                target_path = (md_file.parent / target_file).resolve()
            
            # Determine status - check existence with absolute path
            if target_path.exists():
                if scan_path in target_path.parents or target_path == scan_path:
                    status = 'Valid'
                else:
                    status = 'Outside Root'
                    logging.warning(f"Link outside scan path: {md_file.name} -> {target_file}")
            else:
                status = 'Broken'
                logging.error(f"Broken link: {md_file.name} -> {target_file} (resolved to {target_path})")
            
            # Convert to scan-relative paths for CSV
            source_rel = get_scan_relative_path(md_file, scan_path)
            target_rel = get_scan_relative_path(target_path, scan_path)
            
            # Get or find titles/headers
            title_found = find_markdown_title(content)
            if title_found:
                logging.debug(f"Found header in {md_file.name}: {title_found}")
            
            markdown_header = add_headers_dict(markdown_header, md_file, title_found)

            if markdown_header.get(target_path) is None and target_path.exists():
                with open(target_path, 'r', encoding='utf-8') as tf:
                    tcontent = tf.read()
                ttitle_found = find_markdown_title(tcontent)

                markdown_header = add_headers_dict(markdown_header, target_path, ttitle_found)

            # Add original link
            links_data.append({
                'source_file': source_rel,
                'source_title': markdown_header[md_file],
                'target_file': target_rel,
                'target_title': markdown_header[target_path],
                'link_text': link_text,
                'status': status,
                'hierarchy_level': get_hierarchy_level(md_file, scan_path),
                'link_type': 'original'
            })
            
            # Add backlink entry regardless of validity
            links_data.append({
                'source_file': target_rel,
                'source_title': markdown_header[target_path],
                'target_file': source_rel,
                'target_title': markdown_header[md_file],
                'link_text': '',
                'status': status,  # Use same status as original link
                'hierarchy_level': get_hierarchy_level(target_path, scan_path),
                'link_type': 'backlink'
            })
            
            if status == 'Valid':
                backlinks_map[target_rel].add((source_rel, markdown_header[md_file]))
    
    save_csv_data(csv_path, links_data)
    return backlinks_map


def add_backlinks(scan_path):
    """Add backlinks to markdown files"""
    scan_path = Path(scan_path).resolve()
    backlinks_map = scan_documents(scan_path)
    
    files_updated = 0
    for target_file_rel, source_files_rel in backlinks_map.items():
        # Convert scan-relative path back to absolute for file operations
        if target_file_rel.startswith('/' + scan_path.name):
            rel_part = target_file_rel[len('/' + scan_path.name):].lstrip('/')
            target_path = scan_path / rel_part if rel_part else scan_path
        else:
            target_path = Path(target_file_rel)  # Outside scan path, use as-is
            
        logging.debug(f"Processing backlinks for {target_path.name}")
        
        with open(target_path, 'r', encoding='utf-8') as f:
            content = f.read()
        
        existing_backlinks = get_existing_backlinks(content)
        
        # Remove existing backlinks section
        content = re.sub(r'\n# Backlinks\n.*?(?=\n# |\Z)', '', content, flags=re.DOTALL)
        
        # Build new backlinks section with only new links
        new_backlinks = []
        for source_file_rel in source_files_rel:
            # Convert scan-relative path back to absolute for path calculations
            if False:
                print(source_file_rel)
                if source_file_rel[0].startswith('/' + scan_path.name):
                    rel_part = source_file_rel[0][len('/' + scan_path.name):].lstrip('/')
                    print(rel_part)
                    source_path = scan_path / rel_part if rel_part else scan_path
                    print(rel_part, source_path)
                else:
                    source_path = Path(source_file_rel[0])  # Outside scan path, use as-is
            #source_path = Path(source_file_rel[0])  # Outside scan path, use as-is
            #rel_path = os.path.relpath(source_path, target_path.parent)
            rel_path = source_file_rel[0]
            if rel_path not in existing_backlinks:
                #source_name = source_path.stem
                new_backlinks.append(f'- [{source_file_rel[1]}]({rel_path})')
            #print(source_path, source_path.stem, rel_path, existing_backlinks)
        
        if new_backlinks:
            # Remove existing backlinks section
            content = re.sub(r'\n# Backlinks\n.*?(?=\n# |\Z)', '', content, flags=re.DOTALL)
        
            logging.info(f"Adding {len(new_backlinks)} backlinks to {target_path.name}")
            backlinks_section = '\n# Backlinks\n\n' + '\n'.join(new_backlinks) + '\n'
            
            with open(target_path, 'w', encoding='utf-8') as f:
                f.write(content + backlinks_section)
            files_updated += 1
    
    logging.info(f"Updated {files_updated} files with backlinks")

if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='Generate backlinks for markdown files')
    parser.add_argument('scan_path', nargs='?', help='Folder path to scan for markdown files')
    parser.add_argument('--log-level', choices=['DEBUG', 'INFO', 'WARNING', 'ERROR', 'CRITICAL'], 
                       default='INFO', help='Logging level')
    parser.add_argument('--log-file', help='Log file path (optional)')
    
    args = parser.parse_args()
    
    setup_logging(args.log_level, args.log_file)
    
    # Use SCAN_PATH if defined, otherwise use command line arg or prompt
    if SCAN_PATH:
        scan_path = SCAN_PATH
        logging.info(f"Using hard-coded SCAN_PATH: {SCAN_PATH}")
    else:
        scan_path = args.scan_path or input("Enter scan folder path: ").strip()
    
    try:
        add_backlinks(scan_path)
        logging.info("Backlinks processing completed successfully!")
        print("Backlinks added successfully! Check backlinks.csv for link analysis.")
    except Exception as e:
        logging.error(f"Error processing backlinks: {e}")
        raise